# **RAG del Gemma2 - Muy Simple**

En este cuaderno hemos montado un sistema de RAG muy simple, siguiendo las indicaciones de Gemini que, por cierto, da resultados relativamente aceptables teniendo en cuenta lo sencillo que es.

Lo que es mejor es que se basa en un código perfectamente entendible, a pesar de que hace uso de embaddings y faiss, es decir, todo lo contrario a lo que nos dio DeepSeek. Es una forma de empezar.

Instalamos Unsloth

In [1]:
!pip install unsloth
!pip install --no-deps xformers trl peft accelerate bitsandbytes

Instalamos faiss-cpu, ya que no fue posible hacer lo propio con la versión de gpu.

In [4]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 97.4 MB/s eta 0:00:00


Cargamos el modelo ya fintuneado de 4 bits con Unsloth.

In [5]:
from unsloth import FastLanguageModel
import torch

# La ruta donde están tus 4 ficheros de modelo y el tokenizador
model_path = "/content/drive/MyDrive/Practica_LLM_Engineering_25/modelo-Gemma_2-9b-finetuned-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path, # Unsloth lee la carpeta y une los shards
    max_seq_length = 2048,   # El mismo que usaste en el entrenamiento
    dtype = None,            # Auto-detección (usará float16/bfloat16)
    load_in_4bit = True,     # <--- AQUÍ activamos la magia del 4bit para inferencia
)

# Ponemos el modelo en modo inferencia para que no consuma memoria de gradientes
FastLanguageModel.for_inference(model)

print("¡Modelo Gemma-2 electoral cargado exitosamente en 4 bits!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: If you want to finetune Gemma 2, install flash-attn to make it faster!
To install flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
==((====))==  Unsloth 2026.1.4: Fast Gemma2 patching. Transformers: 4.57.6.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

¡Modelo Gemma-2 electoral cargado exitosamente en 4 bits!


Ahora cargamos el datset original de los datos electorales.

In [6]:
import pandas as pd

In [46]:
strings = {'Sección' : 'str', 'cod_ccaa' : 'str', 'cod_prov' : 'str', 'cod_mun' : 'str', 'cod_sec' : 'str'}

df = pd.read_csv('/content/drive/MyDrive/Practica_LLM_Engineering_25/df_elecciones_19.csv', dtype = strings)

Esta celda hace varias cosas:

- Carga el embedder de la librería transformers
- Prepara una columna en el dataset que reune varios datos de cada fila
- Hace el embedding de cada elemento de esta columna
- Crea el índice FAISS para agilizar las búsquedas

Lo cierto es que en el uso del modelo hubo que potenciar el peso de la fecha de las elecciones, pues el embedding daba preferencia a los municipios, un fenómeno que, según Gemini, dilución semántica.

In [47]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1. Cargamos el modelo que convierte texto en vectores (ligero y potente)
embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 2. Preparamos los textos para indexar
# Creamos una columna que resuma la fila para que el buscador la entienda
# df['resumen_busqueda'] = df.apply(lambda x: f"Municipio: {x['Municipio']}, Elecciones: {x['Elecciones']}, Provincia: {x['Provincia']}", axis=1)

df['resumen_busqueda'] = df.apply(lambda x:
    f"ELECCIONES {x['Elecciones']}. FECHA: {x['Elecciones']}. LUGAR: {x['Municipio']}, {x['Provincia']}. DATOS DE {x['Elecciones']}.",
    axis=1)

# 3. Convertimos a vectores (esto tardará unos minutos por las 72k filas)
embeddings = embedder.encode(df['resumen_busqueda'].tolist(), show_progress_bar=True)


# 4. Creamos el índice FAISS para búsqueda ultra rápida
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))

print("¡Índice RAG creado con éxito!")

Batches:   0%|          | 0/2270 [00:00<?, ?it/s]

¡Índice RAG creado con éxito!


Esta función recoge la pregunta, que pasa por el embedder y FAISS busca las filas más parecidas. Se añade un contexto, y se manda al modelo, que contesta, y la función finalmente devuelve la respuesta parseada.

In [48]:
def sistema_rag_electoral(pregunta, top_k=10):
    # 1. Vectorizar la pregunta del usuario
    query_vector = embedder.encode([pregunta]).astype('float32')

    # 2. Buscar en el espacio vectorial
    distancias, indices = index.search(query_vector, top_k)

    # 3. Recuperar las filas correspondientes del DataFrame original
    filas_recuperadas = df.iloc[indices[0]]

    # 4. Construir el contexto
    contexto = "Datos recuperados del histórico electoral:\n"
    for _, fila in filas_recuperadas.iterrows():
        contexto += f"- En {fila['Municipio']} ({fila['Elecciones']}): "
        contexto += f"porcentage PP: {fila['% PP']:.2%}, porcentage PSOE: {fila['% PSOE']:.2%}, Renta persona 2017: {fila['Renta persona 2017']}€.\n"

    # 5. Crear el prompt final para Gemma-2
    prompt_final = f"""### Instrucción:
{pregunta}

### Contexto:
{contexto}

### Respuesta:"""

    print(prompt_final)

    # 6. Inferencia con tu modelo de 4 bits
    inputs = tokenizer([prompt_final], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split("### Respuesta:")[-1].strip()

Hemos impreso el prompt, y lo cierto es que hay que tener cuidado con la pregunta, porque el embedding es demasiado sencillo. Por ejemplo, hay que poner siempre la palabra 'elecciones' para que reconozca la fecha.

Lo cierto es que los comentarios del modelo parecen tener cierto sentido. No está mal para lo sencillo del sistema de RAG!

In [55]:
resultado = sistema_rag_electoral("Santander elecciones Abril 2019")
print(resultado)

### Instrucción:
Santander elecciones Abril 2019

### Contexto:
Datos recuperados del histórico electoral:
- En Santander (Abril 2019): porcentage PP: 29.01%, porcentage PSOE: 20.44%, Renta persona 2017: 15156.0€.
- En Santander (Abril 2019): porcentage PP: 20.57%, porcentage PSOE: 32.82%, Renta persona 2017: 10181.0€.
- En Santander (Abril 2019): porcentage PP: 20.60%, porcentage PSOE: 32.16%, Renta persona 2017: 10805.0€.
- En Santander (Abril 2019): porcentage PP: 25.61%, porcentage PSOE: 22.97%, Renta persona 2017: 14286.0€.
- En Santander (Abril 2019): porcentage PP: 31.72%, porcentage PSOE: 14.86%, Renta persona 2017: 19087.0€.
- En Santander (Abril 2019): porcentage PP: 31.42%, porcentage PSOE: 19.07%, Renta persona 2017: 15895.0€.
- En Santander (Abril 2019): porcentage PP: 28.95%, porcentage PSOE: 22.38%, Renta persona 2017: 15708.0€.
- En Santander (Abril 2019): porcentage PP: 28.88%, porcentage PSOE: 25.20%, Renta persona 2017: 13962.0€.
- En Santander (Abril 2019): porcenta